<a href="https://colab.research.google.com/github/geovani2311/Data-Cleaning-and-Processing-for-JSON/blob/main/6%20Intelig%C3%AAncia%20Artificial%20Aplicada/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

crie um chatbot usando transformers com o código a seguir que responde perguntas sobre o senac

s=''
while s != 'sair':
  s=input()
  print(s)

para pergunta realizae as seguintes operações NLP:
1) use um modelo Q&A para responder as perguntas
2) mostre o sentimento da pergunta
3) traduza as perguntas para inglês
4) classifique a pergunta do usuário se é relacionada a cursos ou se é relacioada a instituição

In [1]:
!pip install transformers[torch]

In [2]:
# Importa a função 'pipeline', que é a forma mais fácil de usar modelos pré-treinados
from transformers import pipeline

texto_base = """
O Senac (Serviço Nacional de Aprendizagem Comercial) é uma instituição privada sem fins lucrativos que oferece educação profissional e atua na transformação de vidas
através da formação para o trabalho no setor de comércio de bens, serviços e turismo. Fundado em 1946,
ele é financiado por empresários do setor, que contribuem para a sua manutenção.
O Senac oferece cursos livres, técnicos, de graduação e pós-graduação, além de soluções para empresas,
preparando os alunos com metodologias inovadoras e práticas para atuarem com autonomia e atitude crítica no mercado de trabalho
"""

In [3]:
print("Carregando modelos... Este processo pode levar um momento.")

leitor_qa = pipeline("question-answering", model="pierreguillou/bert-base-cased-squad-v1.1-portuguese")

analisador_sentimento = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

tradutor = pipeline("translation", model="unicamp-dl/translation-pt-en-t5")

classificador = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

rotulos_topicos = ["informações de curso", "sobre a instituição", "assunto geral"]

print("="*40)
print("✅ Modelos carregados! O Chatbot está pronto.")
print("Você pode perguntar sobre os cursos ou sobre a instituição.")
print("Digite 'sair' a qualquer momento para terminar.")
print("="*40)

Carregando modelos... Este processo pode levar um momento.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelos carregados! O Chatbot está pronto.
Você pode perguntar sobre os cursos ou sobre a instituição.
Digite 'sair' a qualquer momento para terminar.


In [4]:
while True:
    entrada_do_usuario = input("Você: ")

    if entrada_do_usuario.lower() == 'sair':
        print("🤖 Chatbot: Até a próxima!")
        break

    # Etapa 1: Traduzir a entrada do usuário
    dados_traducao = tradutor(entrada_do_usuario)
    texto_em_ingles = dados_traducao[0]['translation_text']

    # Etapa 2: Classificar o tópico da entrada
    dados_classificacao = classificador(entrada_do_usuario, candidate_labels=rotulos_topicos, hypothesis_template="O tópico deste texto é {}.")
    topico_detectado = dados_classificacao['labels'][0]
    confianca_do_topico = dados_classificacao['scores'][0]

    # Etapa 3: Analisar o sentimento da entrada
    dados_sentimento = analisador_sentimento(entrada_do_usuario)
    sentimento_detectado = dados_sentimento[0]['label']

    # Etapa 4: Buscar a resposta no texto base
    dados_da_resposta = leitor_qa(question=entrada_do_usuario, context=texto_base)
    resposta_encontrada = dados_da_resposta['answer']
    confianca_da_resposta = dados_da_resposta['score']

    # Etapa 5: Exibir todos os resultados
    print("\n🤖 Chatbot (Análise Completa):")
    print("-" * 30)

    if confianca_da_resposta > 0.1:
        print(f'✅ Resposta: "{resposta_encontrada}"')
    else:
        print("❌ Resposta: Desculpe, não encontrei uma resposta para isso no meu texto base.")

    print("\n--- Detalhes Adicionais ---")
    print(f"🔄 Pergunta em Inglês: '{texto_em_ingles}'")
    print(f"🏷️ Tópico Principal: '{topico_detectado}' (Confiança: {confianca_do_topico:.1%})")
    print(f"😊 Sentimento: {sentimento_detectado}")
    print("-" * 30, "\n")

Você: olá o senac parece bom, oque é ele?

🤖 Chatbot (Análise Completa):
------------------------------
❌ Resposta: Desculpe, não encontrei uma resposta para isso no meu texto base.

--- Detalhes Adicionais ---
🔄 Pergunta em Inglês: 'Hello, senac seems good, what is it?'
🏷️ Tópico Principal: 'sobre a instituição' (Confiança: 99.2%)
😊 Sentimento: 3 stars
------------------------------ 

Você: sair
🤖 Chatbot: Até a próxima!
